### Import of required libraries

In [ ]:
import os
import pandas as pd
from tqdm.auto import tqdm
from traffic.core import Traffic, Flight
# from traffic.data import opensky, navaids, airports
from typing import Union, List, Tuple
from traffic.core.mixins import PointMixin
from utils import helperfunctions as hf
import plotly.graph_objects as go

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

### Data fetching

In [ ]:
# Paras
airport = "LSGG"
folder_daily = f"/mnt/beegfs/store/krum/OSN24/{airport}/daily/"
date_range = pd.date_range(start='2023-01-01', end='2023-12-31', freq='D')

# Fetching in daily packages
for date in date_range:
    path = f"{folder_daily}{date.month}/"
    if not os.path.exists(path):
        os.makedirs(path)
    file = path + date.strftime('%Y-%m-%d')+".parquet"
    if not os.path.exists(file):
        print(f"fetching {date}")
        try:
            t = opensky.history(
                start=date.strftime('%Y-%m-%d 00:00'),
                stop=date.strftime('%Y-%m-%d 23:59'),
                bounds=airports[airport].shape.buffer(1).bounds,
            )
            t.to_parquet(file)
        except:
            print(f"failed to fetch {date}")

### Reduction to Landings on RWY04

In [ ]:
# Reduction to landing 04 in monthly packages-----------------------------------
folder_list = os.listdir(folder_daily)
# for each month
for folder in folder_list:
    # List all files in the folder
    file_list = os.listdir(folder_daily + folder)
    filepaths = [os.path.join(folder_daily + folder, file) for file in file_list]

    # Merging to one trafic object
    t = Traffic(
        pd.concat(
            [Traffic.from_file(file).data for file in tqdm(filepaths)],
            ignore_index=True,
        )
    )

    # Cleaning
    t = t.clean_invalid().assign_id().eval(desc="Cleaning", max_workers=20)

    # Reduce to landing 04
    t = (
        t.iterate_lazy()
        .pipe(hf.has_landing_on, "LSGG", "04")
        .eval(desc="landing 04", max_workers=20)
    )

    # Save
    t.to_parquet(
        f"/mnt/beegfs/store/krum/OSN24/LSGG/processed/landing_{folder}.parquet"
    )

# Merge all monthly packages to one yearly package------------------------------
file_list = os.listdir("/mnt/beegfs/store/krum/OSN24/LSGG/processed/")
file_list = [file for file in file_list if file.startswith("landing_")]
filepaths = [
    os.path.join("/mnt/beegfs/store/krum/OSN24/LSGG/processed/", file)
    for file in file_list
]
t = Traffic(
    pd.concat(
        [Traffic.from_file(file).data for file in tqdm(filepaths)],
        ignore_index=True,
    )
)
t.to_parquet(f"/mnt/beegfs/store/krum/OSN24/LSGG/processed/landing_all.parquet")

### Reduction to flights passing BELUS

In [ ]:
t = (
    t.iterate_lazy()
    .pipe(hf.aligned_navpoint, "BELUS")
    .eval(desc="aligned_BELUS", max_workers=20)
)
t.to_parquet(f"/mnt/beegfs/store/krum/OSN24/LSGG/processed/landing_belus.parquet")

### Crop at threshold

In [ ]:
t = (
    t.iterate_lazy()
    .pipe(hf.crop_after_th, "LSGG", "04")
    .eval(desc="Cropping at TH", max_workers=20)
)
t.to_parquet(f"/mnt/beegfs/store/krum/OSN24/LSGG/processed/landing_th.parquet")

### Map-plot

In [ ]:
t = Traffic.from_file(f"/mnt/beegfs/store/krum/OSN24/LSGG/processed/landing_th.parquet")
t = (
    t.iterate_lazy()
    .pipe(hf.crop_before_wp, "BELUS")
    .eval(desc="Cropping before TH", max_workers=20)
)

In [ ]:
# Create a single figure for the map
fig = go.Figure()

# Add flight trajectories
for flight in t.sample(100):
    fig.add_trace(
        go.Scattermapbox(
            mode="lines",
            lat=flight.data["latitude"],
            lon=flight.data["longitude"],
            line=dict(width=1.5, color="#757ef3"),
            opacity=0.5,
            name="STAR",
            showlegend=False,
        )
    )

# Add BELUS navaid
fig.add_trace(
    go.Scattermapbox(
        mode="markers+text",
        lat=[navaids["BELUS"].latitude],
        lon=[navaids["BELUS"].longitude],
        marker=dict(size=10, color="red"),
        text=["BELUS"],
        textposition="middle right",
        textfont=dict(color="red", size=18),
        name="BELUS",
        legendgroup="Map",
    )
)

# Update layout for the map
fig.update_layout(
    width=2000,
    height=1000,
    margin=dict(l=50, r=0, t=40, b=50),
    mapbox=dict(
        style="carto-positron",
        zoom=8,
        center=dict(
            lat=46.23696078946708,
            lon=6.109110255543277,
        ),
    ),
)

# Show the figure
fig.show()


In [ ]:
# Empty lists
ids = []
distances = []
timestamp = []

for flight in tqdm(t):
    try:
        # determine distance form BELUS to RWY04 TH
        distance = (
            hf.crop_before_wp(flight, "BELUS")
            .cumulative_distance()
            .data.cumdist.iloc[-1]
        )
        # Append distance and is to lists
        distances.append(distance)
        ids.append(flight.flight_id)
        timestamp.append(flight.start)
    except:
        print(f"Nothing after BELUS for {flight.flight_id}")

# Turn lists into dataframe
df = pd.DataFrame(
    {"flight_id": ids, "distance": distances, "timestamp": timestamp}
)

In [ ]:
fig = go.Figure()

fig.add_trace(
    go.Histogram(
        x=df.distance,
        xbins=dict(
            start=0,
            end=int(max(df.distance)),
            size=0.5,
        ),
        marker=dict(color="#1f77b4"),
        name="Observed distances",
        showlegend=True,
    ),
)

fig.add_shape(
    type="line",
    x0=95,
    x1=95,
    y0=0,
    y1=1,
    yref="paper",
    line=dict(
        color="#d62728",
        width=3,
    ),
)

# For proper legend and axes
fig.add_trace(
    go.Scatter(
        x=[
            95,
            95,
        ],
        y=[0, 1],
        mode="lines",
        line=dict(color="#d62728", width=3),
        name="Distance according to STAR",
        showlegend=True,
        legendgroup="Histogram",
    )
)

# Set x-axis range between 0 and 100
fig.update_xaxes(
    range=[25, 100],
    title_text="Distance flown from BELUS to THR RWY04 [NM]",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

fig.update_yaxes(
    title_text="Count",
    titlefont=dict(size=30),
    tickfont=dict(size=25),
    title_standoff=30,
)

fig.update_layout(
    width=2000,
    height=800,
    margin=dict(l=50, r=0, t=40, b=50),
    legend=dict(
        font=dict(size=30),
        yanchor="top",
        xanchor="right",
        x=0.9,  # Positioning of the legend
        y=0.95,
        tracegroupgap=10  # Increase space between legend groups (items)
    ),
)